In [1]:
import numpy as np 
import pandas as pd
!pip install jsonlines
import numpy as np
import pandas as pd
import jsonlines
import nltk
import csv
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


You are using pip version 19.0, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to /home/aditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def termFrequency(sentence):
    wordlist=word_tokenize(sentence)
    wordfreq = [wordlist.count(p) for p in wordlist]
    freqDict = dict(list(zip(wordlist,wordfreq)))
    aux = [[key,freqDict[key]] for key in freqDict]
    #aux.sort()
    #aux.reverse()
    return freqDict
sortedDict = termFrequency("They are the third such group to graduate since the program's inauguration in 1998.\n\n")
#for s in sortedDict: print(str(s))
print(sortedDict)

{'They': 1, 'are': 1, 'the': 2, 'third': 1, 'such': 1, 'group': 1, 'to': 1, 'graduate': 1, 'since': 1, 'program': 1, "'s": 1, 'inauguration': 1, 'in': 1, '1998': 1, '.': 1}


In [3]:
def getSentenceCount(document, word):
    ct=0
    for i in range(0,len(document)):
        wordList = word_tokenize(document[i])
        if wordList.count(word)>0:
            ct+=1
    return ct
# count = getSentenceCount(sentences, "children")
# print(count)

In [4]:
def inverseTermFrequency(document, sent1,sent2):
    d=len(document)
    a = word_tokenize(sent1) 
    b = word_tokenize(sent2)
    sent = list(set(a) & set(b))
    sentCount = [getSentenceCount(document,word) for word in sent]
    keyList = sent
    # print(keyList)
    idfDict = dict(list(zip(keyList,sentCount)))
    #freqDict,aux = termFrequency(sentence)
    #sentCount = [getSentenceCount(document, key[0]) for key inD freqDict]
    #keyList = [key[0] for key in freqDict]
    #idfDict = dict(list(zip(keyList, sentCount)))
    #aux = [[key,idfDict[key]] for key in idfDict]
    #aux.sort()
    #aux.reverse()
    return idfDict
# sortedDict = inverseTermFrequency(sentences,  'Bernstein, 41, and her husband, Doug Flaumenbaum, 33, both recovering crack and heroin addicts, were among three dozen men and women who regained custody of their children.')
#for key in sortedDict: print(sortedDict[key])
# print(sortedDict)


In [5]:
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
stop_words=stopwords.words('english')
def sentence_similarity(sentences,sent1,sent2, stopwords=None):
    # print("here")
    a = word_tokenize(sent1) 
    b = word_tokenize(sent2)
    sent = list(set(a) & set(b))
    termFrequency1 = termFrequency(sent1)
    termFrequency2 = termFrequency(sent2)
    idf = inverseTermFrequency(sentences, sent1,sent2)
    # print(idf.keys())
    # print(sent)
    sum1 = 0
    for word in sent:
        a = 0
        #if(word in stop_words):
         # continue
        # if word in termFrequency1.keys():
        a = termFrequency1[word]
        b = 0
        # if word in termFrequency2.keys():
        b = termFrequency2[word]
        c = np.log(len(sentences)/idf[word])
        sum1+= a*b*c*c
    sum2 = 0
    for word in sent:
        a = termFrequency1[word]
        b = np.log(len(sentences)/idf[word])
        sum2+= a*b*b
    sum3 = 0
    for word in sent:
        a = termFrequency2[word]
        b = np.log(len(sentences)/idf[word])
        sum3+= a*b*b
    return sum1/(np.sqrt(sum2)*np.sqrt(sum3))
    #return 0
    
# simi = sentence_similarity( 'Bernstein, 41, and her husband, Doug Flaumenbaum, 33, both recovering crack and heroin addicts, were among three dozen men and women who regained custody of their children.' , "They are the third such group to graduate since the program's inauguration in 1998.\n\n")
# print(simi)

In [6]:
stop_words=stopwords.words('english')
import math
def build_similarity_matrix(sentences, stopwords=None):
    
    S= np.zeros((len(sentences), len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            #print(idx1,idx2)
            if idx1 == idx2:
                continue

            t = sentence_similarity(sentences,sentences[idx1], sentences[idx2], stop_words)
            if(math.isnan(t)):
              S[idx1][idx2]=0
            else:
              S[idx1][idx2]=t
            #print(S[idx1][idx2])
    for idx in range(len(S)):
        # np.seterr(divide='ignore', invalid='ignore')
        S[idx] /= 1+S[idx].sum()
    # print("here")
    return S

In [7]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        # np.seterr(divide='ignore', invalid='ignore')
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P

In [8]:
def textrank(sentences, top_n=5,stopwords=None):
    
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary

In [9]:
# import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from operator import itemgetter
path = "train-stats.jsonl"
data = []
data_sum = []


cnt = 0
with jsonlines.open(path) as f:

    for line in f.iter():
        sample =  line
        cnt = cnt+1
        if(cnt%1000==0):
          print(cnt)
        if(cnt==100000):
            break
        
        data.append(line['text'])
        data_sum.append(line['summary'])
data[78]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


'Constitution USA With Peter Sagal 9 p.m., Channel 2\n\nYou’ve probably heard him on NPR’s “Wait Wait … Don’t Tell Me,” where he is endlessly witty. Now you can see him, and his significant eyebrows, on this new four-part series. Sagal takes a cross-country motorcycle trip to explore the contemporary relevance of the Constitution. Among the issues he addresses with the people he meets: same-sex marriage, presidential power, the separation of church and state, medical marijuana, gun control, and health care.\n\nThree of my favorites are on tonight, so I thought I would foist that information on you. First is Edie Falco, whose “Nurse Jackie” is so steadily engaging and honest. Then, Marc Maron pays his almost 50th visit to Conan, to promote his amusing new IFC comedy. And finally, Iron & Wine, led by Sam Beam, performs from their new album. While the new sound is too jazzy and soft-rocky for me, his trademark low-fi banging-acoustic sound on “Woman King,” “The Shepherd’s Dog,” and “Aroun

In [10]:
!pip install rouge-score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_precision = 0
rouge1_recall = 0
rouge1_f = 0
rougeL_precision = 0
rougeL_recall = 0
rougeL_f = 0
count = 0
for i in range(9000,10000):
      print(i)
      text = data[i]
      temp = sent_tokenize(text)
      sentences = []
      for sent in temp:
        sentences.append(sent)
      k = 1
      if(len(sentences)<=20):
        k = 5
      elif(len(sentences)<=40):
        k=10
      elif(len(sentences)<=60):
        k=15
      elif(len(sentences)<=80):
        k=20
      if len(sentences)>80:
        continue
      count = count + 1
      generated_summ = textrank(sentences,top_n=k,stopwords=stopwords.words('english'))
      text_summ = data_sum[i]
      generated = ""
      for i in range(0,len(generated_summ)):
        generated = generated + generated_summ[i]
      scores = scorer.score(generated,text_summ)
      rouge1_precision = rouge1_precision + scores['rouge1'].precision
      rouge1_recall = rouge1_recall + scores['rouge1'].recall
      rouge1_f = rouge1_f + scores['rouge1'].fmeasure
      rougeL_precision = rougeL_precision + scores['rougeL'].precision
      rougeL_recall = rouge1_recall + scores['rougeL'].recall
      rougeL_f = rougeL_f + scores['rougeL'].fmeasure
      print([rouge1_precision,rouge1_recall,rouge1_f,rougeL_precision,rougeL_recall,rougeL_f])

You are using pip version 19.0, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
9000


/home/aditya/Environments/python_ml/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


[0.7257142857142858, 0.41776315789473684, 0.5302713987473904, 0.56, 0.7401315789473684, 0.4091858037578288]
9001
[1.3349096880131364, 0.5986505299083887, 0.809218767168443, 0.9163218390804598, 0.7044525776899587, 0.5723436984946709]
9002
[1.9802997589351223, 1.0536505299083887, 1.342943107344396, 1.370222548300318, 1.3736505299083888, 0.9477102674096269]
9003
[2.622501593797508, 1.197387696232824, 1.5778424362034564, 1.8105895207773823, 1.2959503245695796, 1.1087840929129826]
9004
[3.481197245971421, 1.670441588448393, 2.1878810462420666, 2.62580691208173, 2.1195433848555787, 1.6879346720635617]
9005
[4.047863912638087, 2.45747862548543, 2.846795774924237, 3.172473578748397, 3.2167378847446892, 2.323593586792244]
9006
[4.5444625520938695, 2.610198290757397, 3.080395774924237, 3.41737153793207, 2.6855120982887777, 2.438793586792244]
9007
[5.308437707373373, 3.0161388848168027, 3.61056818871734, 3.9763777491121943, 3.3131685877870996, 2.8267246212750026]
9008
[6.195655752486155, 3.296423

[47.551660328383356, 16.120603756795816, 22.598490020190727, 36.618250253655695, 16.138216281257655, 17.280116634731918]
9068
[48.00999366171669, 16.213040731585732, 22.75233617403688, 36.95158358698903, 16.280267622342034, 17.39200474662003]
9069
[48.4174010691241, 16.294522213067214, 22.888138643172685, 37.17380580921125, 16.33896665751166, 17.466078820694104]
9070
[49.195178846901875, 16.42064833919334, 23.105192906738576, 37.67380580921125, 16.501729420274422, 17.605613704415035]
9071
[49.695178846901875, 16.446180254086958, 23.153775902689993, 38.09047247587792, 16.46745684983164, 17.64609953437455]
9072
[50.45379953655705, 16.53765634556305, 23.31704120881244, 38.504265579326194, 16.5875523954591, 17.735153337714067]
9073
[51.41379953655705, 16.75583816374487, 23.672596764367995, 39.464265579326195, 16.974019981926688, 18.09070889326962]
9074
[51.91379953655705, 16.798262406169112, 23.75080905487079, 39.82140843646905, 16.828565436472143, 18.14657481505733]
9075
[52.6637995365570

[87.4854906155187, 21.8666386218483, 32.406296159933206, 67.20246157990378, 21.925462151260064, 24.793868294184655]
9136
[87.75215728218537, 21.919971955181634, 32.495185048822094, 67.40246157990379, 21.959971955181633, 24.86053496085132]
9137
[88.35215728218536, 21.963034156138573, 32.57554219167924, 67.86912824657045, 21.99652697910508, 24.92303496085132]
9138
[88.89382394885203, 22.007554704083777, 32.6578206726919, 68.16079491323713, 22.031527306823502, 24.967338758319677]
9139
[89.6630547180828, 22.131011494207232, 32.8705866301387, 68.85310260554482, 22.242122605318343, 25.158828120021806]
9140
[90.1630547180828, 22.223448468997148, 33.02661499893303, 69.35310260554482, 22.315885443787064, 25.314856488816133]
9141
9142
[90.9487690037971, 22.30258515964463, 33.17040584860623, 69.78167403411625, 22.34575062727053, 25.393287861365152]
9143
[91.4487690037971, 22.32602265964463, 33.21518196800921, 70.17056292300514, 22.344251826311297, 25.42811373201192]
9144
[92.02771637221815, 22.40

...

[432.2697929146479, 67.41360699708467, 106.52576239183972, 334.4334908097905, 67.4311508567338, 83.21420265536186]
9694
[432.91979291464787, 67.459381644972, 106.6112887076292, 334.9334908097905, 67.49459291257763, 83.27999212904606]
9695
[433.19757069242564, 67.59451678010713, 106.79310688944739, 335.21126858756827, 67.72965191524226, 83.46181031086425]
9696
[433.89757069242563, 67.67115911587355, 106.93126478418422, 335.71126858756827, 67.725903641421, 83.56049452139057]
9697
[434.4531262479812, 67.82579829113128, 107.17320026805518, 336.0446019209016, 67.91858179628592, 83.70565581171314]
9698
[434.9692552802392, 68.02332915532881, 107.45891455376947, 336.36718256606287, 68.14678594545227, 83.88422724028457]
9699
[435.1359219469059, 68.02864830426498, 107.46922383212, 336.53384923272955, 68.03396745320114, 83.89453651863509]
9700
[435.76750089427435, 68.06974419467593, 107.54639425012643, 336.9549018643085, 68.0971414549499, 83.94598346397271]
9701
[436.35724448401794, 68.1137212501

[476.4335919949998, 76.04404134676673, 119.99643230287533, 368.03087063019785, 76.07693608360884, 93.7234120953678]
9763
[476.98531613293085, 76.1044187052573, 120.1052758402903, 368.4101809750254, 76.14592813921956, 93.79824202734059]
9764
[477.5307706783854, 76.14667222638407, 120.18370721283932, 368.81927188411635, 76.17836236722914, 93.85706555675236]
9765
[478.4003358957767, 76.32849040820226, 120.48445909253857, 369.3844892754207, 76.44667222638408, 94.05255427855687]
9766
[479.262404861294, 76.4897807307829, 120.75619822297335, 369.93621341335177, 76.59300653723452, 94.22646732203513]
9767
[480.012404861294, 76.60007484842995, 120.94850591528105, 370.38621341335175, 76.66625131901819, 94.34185193741975]
9768
[481.012404861294, 76.75224876147344, 121.21265685867728, 371.3147848419232, 76.89355310929953, 94.58713495628767]
9769
[481.89475780247045, 76.87936740554123, 121.4348790808995, 372.02066719486436, 76.98106232079547, 94.76491273406545]
9770
[482.54475780247043, 76.908712778

[521.7136940916955, 85.01522112130971, 133.76752993510024, 403.32082153067984, 85.34610347425088, 104.49205809172041]
9836
[522.3082886862901, 85.10428994722064, 133.92245951256504, 403.80730801716635, 85.17716444114777, 104.6188186551007]
9837
[523.3082886862901, 85.26611567336171, 134.20103094113648, 404.80730801716635, 85.42794139950279, 104.89739008367212]
9838
[523.7368601148615, 85.28296960594598, 134.23346337356892, 405.2358794457378, 85.29982353853025, 104.92982251610455]
9839
[524.1768601148616, 85.33229696020607, 134.32217305098828, 405.51587944573777, 85.36368709473521, 104.98627412900778]
9840
[525.1768601148616, 85.49896362687274, 134.60788733670256, 406.51587944573777, 85.66563029353941, 105.27198841472207]
9841
[526.1485582280692, 85.90130737687274, 135.17694811018322, 407.4875775589453, 86.30365112687274, 105.84104918820273]
9842
[526.7991606377077, 86.07005737687274, 135.4449381846249, 407.78878237822244, 86.14818237687274, 105.96511866711091]
9843
[527.4658273043743, 

[570.8791946165794, 99.00337021574573, 152.6894542197442, 443.89642175644906, 99.01799009878667, 120.42681729416077]
9907
[571.7052815731012, 99.05147148156851, 152.78036331065329, 444.5920739303621, 99.09197781068244, 120.50337231808422]
9908
[572.2767101445297, 99.0830920349282, 152.84028840428624, 444.94921678750495, 99.102854880778, 120.54082550160481]
9909
[573.2599034218406, 99.34194159245033, 153.2500957598029, 445.90719998078225, 99.59415398183086, 120.94012497621077]
9910
[573.90696224537, 99.66786751837626, 153.6835932967487, 446.40719998078225, 99.91971937022811, 121.27510034566889]
9911
[574.2680733564811, 99.71223611905884, 153.76262065237182, 446.68497775856, 99.74636581189161, 121.33589061922513]
9912
[574.7680733564811, 99.73626014308287, 153.80846592457812, 447.12247775856, 99.7572811641039, 121.37600523240565]
9913
[575.3589824473902, 99.77888309390255, 153.88797662794204, 447.44065957674184, 99.80183391357468, 121.41881868806314]
9914
[575.9214824473902, 99.813902549

[612.2314622129236, 103.98841559877415, 160.97547078683493, 475.136736351136, 104.11368099155547, 126.89909353020737]
9974
[612.8981288795902, 104.08198285023614, 161.1395733509375, 475.636736351136, 104.15215828883264, 127.02217045328429]
9975
[613.3624145938759, 104.14763941589271, 161.2546175987251, 476.02959349399316, 104.20319497144827, 127.11951558602765]
9976
[613.7624145938759, 104.19525846351176, 161.33972398170383, 476.3295934939932, 104.23097274922605, 127.18334537326169]
9977
[614.2169600484214, 104.25623407326786, 161.44725086342424, 476.69322985762955, 104.30501456107274, 127.26936687863804]
9978
[614.931245762707, 104.28979112024773, 161.5113534275268, 477.19322985762955, 104.31328105313364, 127.31423867350983]
9979
[615.6098171912785, 104.34227730809303, 161.60878932496271, 477.5860870004867, 104.37266404842453, 127.3706489299201]
9980
[616.4669600484214, 104.47561064142637, 161.83955855573194, 478.40751557191527, 104.60338841920415, 127.59180277607393]
9981
9982
[617.1

In [11]:
csvRow = [rouge1_precision/count,rouge1_recall/count,rouge1_f/count,rougeL_precision/count,rougeL_recall/count,rougeL_f/count]
with open('summary-stats.csv', 'a') as fd:
    writer = csv.writer(fd)
    writer.writerow(csvRow)